In [1]:
# Code written by: Michael Bramble | michael.s.bramble@jpl.nasa.gov
# unmixing test script for EMIT AMD investigation
# copying process from my CRISM/VIR studies
# 20240124 - initial version

# general process
# (pull image info form a list and being a loop)
# load reflectance image and geometry file (phocube)
# load endmembers for unmixing (endmember script massages lab spectra to match instrument, also ssa?)
# subset data to relevant wavelenth range
# calculate mean-column endmember?
# begin nested loop through each line and sample
# pull the pixel's reflectance specturm (convert to ssa?)
# perform linear least squares unmixing, return fractions and rmse
# save maps of spectral fraction and rmse
# save visualizations
# (end one image and loop iteration)

In [1]:
# import earthaccess
import os
# from osgeo import gdal
import numpy as np
import pandas as pd
import xarray as xr
from scipy.optimize import lsq_linear
# import math
# import rasterio as rio
# import holoviews as hv
import hvplot.xarray
# import netCDF4 as nc
import sys
sys.path.append('/Users/bramble/My Drive/_JPL_AMD/EMIT-Data-Resources-main/python/modules/')
from emit_tools import emit_xarray

In [2]:
# load reflectance image
granule_asset_id = 'EMIT_L2A_RFL_001_20231006T174849_2327912_017.nc'
fp = f'/Users/bramble/Documents/emit/leadville/ref/{granule_asset_id}'
# load data set and remove the bad bands
ds_geo = emit_xarray(fp, ortho=True)
ds_geo['reflectance'].data[:,:,ds_geo['good_wavelengths'].data==0] = np.nan

# as a test, plot spectra taken from the georeferenced image
point = ds_geo.sel(longitude=-106.5,latitude=39,method='nearest')
ds_geo.sel(wavelengths=850, method='nearest').hvplot.image(cmap='viridis', aspect = 'equal', rasterize=True) +\
point.hvplot.line(y='reflectance',x='wavelengths', color='black', frame_width=400).opts(
    title = f'Latitude = {point.latitude.values.round(3)}, Longitude = {point.longitude.values.round(3)}')

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


BokehModel(combine_events=True, render_bundle={'docs_json': {'b83243c8-45b4-467a-8312-7296ff113bad': {'version…

Task exception was never retrieved
future: <Task finished name='Task-7' coro=<Callback.process_on_change() done, defined at /Users/bramble/opt/miniconda3/envs/emit/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py:322> exception=UnsetValueError("figure(id='p1010', ...).inner_height doesn't have a value set")>
Traceback (most recent call last):
  File "/Users/bramble/opt/miniconda3/envs/emit/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py", line 340, in process_on_change
    msg[attr] = self.resolve_attr_spec(path, cb_obj)
  File "/Users/bramble/opt/miniconda3/envs/emit/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py", line 248, in resolve_attr_spec
    resolved = getattr(resolved, p, None)
  File "/Users/bramble/opt/miniconda3/envs/emit/lib/python3.9/site-packages/bokeh/core/property/descriptors.py", line 283, in __get__
    raise UnsetValueError(f"{obj}.{self.name} doesn't have a value set")
bokeh.core.property.descriptors.UnsetValue

In [3]:
# SPATIAL SUBSET IMAGE HERE 
# FOR NOW, MANUALLY. IN FUTURE, AUTOMATE

# LEADVILLE
max_lat = 39.3
min_lat = 39.2
max_lon = -106.3
min_lon = -106.2

# simplify the ref band
band = ds_geo.reflectance

# select just the spatial subset around the mine
subset_data = band.sel({'latitude' : slice(max_lat, min_lat),
                      'longitude' : slice(max_lon, min_lon)})

# GET LAT/LON ARRAYS
band_lon =  subset_data['longitude']
band_lat =  subset_data['latitude']

# REPALCE NaNs. Likely not needed in the final version.
subset_data = subset_data.fillna(0)

# # DISPLAY SUBSET DATA
# subset_data.sel(wavelengths=850, method='nearest').hvplot.image(cmap='viridis', frame_width=500, geo=True, tiles='EsriImagery').opts(
#     xlabel=f'{ds_geo.longitude.long_name} ({ds_geo.longitude.units})', ylabel=f'{ds_geo.latitude.long_name} ({ds_geo.latitude.units})')

# PLOT SPECTRUM FROM SPATIAL SUBSET DATA USING INDEXES
subset_data[50,50].hvplot.line(y='reflectance',x='wavelengths', color='black')

:Curve   [wavelengths]   (reflectance)

In [17]:
# load wavelenth array
file = '/Users/bramble/My Drive/_JPL_AMD/endmembers/emit_wavelengths.txt'
wavelengths_emit_nm = pd.read_csv(file,header=None)
wavelengths_emit_um = wavelengths_emit_nm[0]/1000

# load endmember library
from script_py_load_amd_endmembers import endmembers
endmember_array = np.arange(1,endmembers.shape[1]+1,1)
endmembers_xr = xr.DataArray(endmembers, coords=[("wavelengths", wavelengths_emit_um), ("endmembers", endmember_array)])
endmembers_xr.sel(endmembers=[5, 6]).hvplot.line(x='wavelengths')


BokehModel(combine_events=True, render_bundle={'docs_json': {'80170466-3a9a-4ad6-858a-ab3e2dfb3634': {'version…

In [5]:
# perform spectral subset on EMIT image and endmember array


# test = subset_data.sel(wavelengths=slice(500, 800))
# test2 = endmembers_xr.sel(wavelengths=slice(0.5, 0.8))



In [26]:
# test = subset_data[50,50]
# subset_data.coords['latitude']
# subset_data.coords['longitude']
# subset_data
# newarray = xr.DataArray(None, coords=dict(latitude=subset_data.coords['latitude'],longitude=subset_data.coords['longitude']),dims=[subset_data.dims[0],subset_data.dims[1]])
# newarray

# import matplotlib.pyplot as plt

# test[50,50].hvplot.line(y='reflectance',x='wavelengths', color='black') + 
# test2.hvplot.line('wavelengths')

# test2.sel(endmembers=[2,3]).hvplot.line()
# test2
test
# plt.plot(wavelengths_emit_um,endmembers)



<xarray.DataArray 'reflectance' (latitude: 185, longitude: 185, wavelengths: 40)>
array([[[0.01840826, 0.01963402, 0.02225847, ..., 0.12930691,
         0.13057142, 0.13182637],
        [0.02269246, 0.02467454, 0.02740713, ..., 0.13737631,
         0.13881435, 0.14025141],
        [0.02269246, 0.02467454, 0.02740713, ..., 0.13737631,
         0.13881435, 0.14025141],
        ...,
        [0.02493739, 0.02609637, 0.0283846 , ..., 0.12794207,
         0.12947984, 0.13099997],
        [0.02493739, 0.02609637, 0.0283846 , ..., 0.12794207,
         0.12947984, 0.13099997],
        [0.02950149, 0.03098432, 0.03254041, ..., 0.12752992,
         0.12902942, 0.13053037]],

       [[0.01840826, 0.01963402, 0.02225847, ..., 0.12930691,
         0.13057142, 0.13182637],
        [0.01828144, 0.01970847, 0.02161439, ..., 0.12790802,
         0.12915584, 0.13038702],
        [0.02252404, 0.02409226, 0.02648886, ..., 0.13133119,
         0.13269606, 0.13405518],
...
        [0.02699075, 0.02858379, 0.02932964, ..., 0.1101843 ,
         0.11153231, 0.11288594],
        [0.03254488, 0.03415476, 0.03538474, ..., 0.11819359,
         0.11962022, 0.12105937],
        [0.03254488, 0.03415476, 0.03538474, ..., 0.11819359,
         0.11962022, 0.12105937]],

       [[0.07568347, 0.07774449, 0.08045158, ..., 0.17980689,
         0.18157673, 0.18328033],
        [0.07568347, 0.07774449, 0.08045158, ..., 0.17980689,
         0.18157673, 0.18328033],
        [0.06157585, 0.0629418 , 0.06525332, ..., 0.1431515 ,
         0.14489862, 0.14660592],
        ...,
        [0.02214249, 0.02329728, 0.02362474, ..., 0.09971824,
         0.10064368, 0.1015616 ],
        [0.02214249, 0.02329728, 0.02362474, ..., 0.09971824,
         0.10064368, 0.1015616 ],
        [0.02953616, 0.03113393, 0.03166846, ..., 0.10799786,
         0.10917006, 0.11034097]]], dtype=float32)
Coordinates:
  * latitude          (latitude) float64 39.3 39.3 39.3 39.3 ... 39.2 39.2 39.2
  * longitude         (longitude) float64 -106.3 -106.3 -106.3 ... -106.2 -106.2
  * wavelengths       (wavelengths) float32 507.2 514.7 522.1 ... 790.4 797.9
    good_wavelengths  (wavelengths) float32 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    fwhm              (wavelengths) float32 8.435 8.436 8.438 ... 8.488 8.489
    elev              (latitude, longitude) float32 3.277e+03 ... 3.562e+03
    spatial_ref       int64 0
Attributes:
    long_name:  Surface Reflectance
    units:      unitless

In [100]:
# developing empty array for the model outputs

# endmember_array = np.arange(1,endmembers.shape[1]+1,1)
# subset_data.coords['latitude']

# endmembers_coords = [("endmembers", np.arange(1,endmembers.shape[1]+1,1))]
# endmember_array = xr.DataArray(np.arange(1,endmembers.shape[1]+1,1),endmembers_coords,dims=["endmembers"])

# newarray_res_x = xr.DataArray(None, coords=dict(latitude=subset_data.coords['latitude'],longitude=subset_data.coords['longitude'],endmembers=endmember_array.coords['endmembers']),dims=[subset_data.dims[0],subset_data.dims[1],'endmembers'])
# newarray_res_x


<xarray.DataArray (latitude: 185, longitude: 185)>
array([[None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       ...,
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None]], dtype=object)
Coordinates:
  * latitude   (latitude) float64 39.3 39.3 39.3 39.3 ... 39.2 39.2 39.2 39.2
  * longitude  (longitude) float64 -106.3 -106.3 -106.3 ... -106.2 -106.2 -106.2

In [107]:
# begin unmixing process


# newarray = xr.DataArray(None, subset_data.coords, subset_data.dims)
newarray_res_x = xr.DataArray(None, coords=dict(latitude=subset_data.coords['latitude'],longitude=subset_data.coords['longitude'],endmembers=endmember_array.coords['endmembers']),dims=[subset_data.dims[0],subset_data.dims[1],'endmembers'])
newarray_res_opt = xr.DataArray(None, coords=dict(latitude=subset_data.coords['latitude'],longitude=subset_data.coords['longitude']),dims=[subset_data.dims[0],subset_data.dims[1]])
for i in np.arange(0,len(band_lon)-1):
    for j in np.arange(0,len(band_lat)-1):
        spectrum = subset_data[i,j]

        res = lsq_linear(endmembers, spectrum, bounds=(0, 1), lsmr_tol='auto', verbose=0)

        res.x[res.x < 0.0001] = 0

        newarray_res_x[i,j] = res.x

        newarray_res_opt[i,j] = res.optimality






In [108]:

print(newarray_res_x[20,20])

print(newarray_res_opt[20,20])


<xarray.DataArray (endmembers: 8)>
array([0.13836079622132716, 0.13836079614990163, 0.0, 0.0, 0.0,
       0.045015827844514904, 0.1950437889875802, 0.0], dtype=object)
Coordinates:
    latitude    float64 39.29
    longitude   float64 -106.3
  * endmembers  (endmembers) int64 1 2 3 4 5 6 7 8
<xarray.DataArray ()>
array(5.668653084780885e-12, dtype=object)
Coordinates:
    latitude   float64 39.29
    longitude  float64 -106.3


In [10]:
# import numpy as np
# from scipy.sparse import rand
# from scipy.optimize import lsq_linear
# rng = np.random.default_rng()

# m = 20000
# n = 10000

# A = rand(m, n, density=1e-4, random_state=rng)
# b = rng.standard_normal(m)

# lb = rng.standard_normal(n)
# ub = lb + 1

# res = lsq_linear(A, b, bounds=(lb, ub), lsmr_tol='auto', verbose=1)

The relative change of the cost function is less than `tol`.
Number of iterations 16, initial cost 1.4897e+04, final cost 1.0982e+04, first-order optimality 3.90e-08.


In [19]:
# res
# res.x
# res.fun
# res.optimality
# res.unbounded_sol

3.89634722357619e-08